# Finetune on experimental data
This notebook demonstrates how to finetune METL models on experimental data.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys

# define the name of the project root directory
project_root_dir_name = "metl"

# find the project root by checking each parent directory
current_dir = os.getcwd()
while os.path.basename(current_dir) != project_root_dir_name and current_dir != os.path.dirname(current_dir):
    current_dir = os.path.dirname(current_dir)

# change the current working directory to the project root directory
if os.path.basename(current_dir) == project_root_dir_name:
    os.chdir(current_dir)
else:
    print("project root directory not found")
    
# add the project code folder to the system path so imports work
module_path = os.path.abspath("code")
if module_path not in sys.path:
    sys.path.append(module_path)

# Acquire an experimental dataset

For demonstration purposes, this repository contains the [avGFP dataset](../data/dms_data/avgfp) from [Sarkisyan et al. (2016)](https://doi.org/10.1038/nature17995). 
See the [metl-pub](https://github.com/gitter-lab/metl-pub) repository to access the other experimental datasets we used in our preprint.
See the README in the [dms_data](../data/dms_data) directory for information about how to use your own experimental dataset. 

# Acquire a pretrained model
Pretrained METL models are available in the [metl-pretrained](https://github.com/gitter-lab/metl-pretrained) repository. You can use one of those, or you can pretrain your own METL model (see [pretraining.ipynb](pretraining.ipynb)). 

For demonstration purposes, we include a pretrained avGFP METL-Local model from the [metl-pretrained](https://github.com/gitter-lab/metl-pretrained) repository in the [pretrained_models](../pretrained_models) directory. This model is `METL-L-2M-3D-GFP` (UUID: `Hr4GNHws`).
It is the avGFP METL-Local source model we used for the analysis in our preprint.

We will show how to finetune this model using the [experimental avGFP dataset](../data/dms_data/avgfp).

# Training arguments

The script for finetuning on experimental data is [train_target_model.py](train_target_model.py). This script has a number of arguments you can view by uncommenting and running the below cell. There are additional arguments related to architecture that won't show up if you run the command, but you can view them in [models.py](../code/models.py) in the `TransferModel` class.

In [3]:
# !python code/train_target_model.py -h

We set up finetuning arguments for this example in [finetune_avgfp_local.txt](../args/pretrain_avgfp_local.txt) in the [args](../args) directory. This argument file can be used directly with [train_target_model.py](train_target_model.py) by calling the command `!python code/train_target_model.py @args/finetune_avgfp_local.txt` (we do this in the next section).

Uncomment and run the cell below to view the contents of the argument file. The sections below will walk through and explain the key arguments.

In [4]:
# with open("args/finetune_avgfp_local.txt", "r") as file:
#     contents = file.read()
#     print(contents)

## Dataset arguments


Specify the dataset name and the train/val/test split. The dataset must be defined in [datasets.yml](../data/dms_data/datasets.yml). For demonstration purposes, we are using one of the reduced dataset size splits with a dataset size of 160 (train size of 128).
```
--ds_name
avgfp
--split_dir
data/dms_data/avgfp/splits/resampled/resampled_ds160_val0.2_te0.1_w1abc2f4e9a64_s1_r8099/resampled_ds160_val0.2_te0.1_w1abc2f4e9a64_s1_r8099_rep_0
```

Specify the names of the train, validation, and test set files in the split directory. Using "auto" for the test_name will select the super test set ("stest.txt") if it exists in the split directory, otherwise it will use the standard test set ("test.txt").

```
--train_name
train
--val_name
val
--test_name
test
```

The name of the target column in the dataset dataframe. The model will be finetuned to predict the score in this column.

```
--target_names
score
```

The METL-Local model we are finetuning uses 3D structure-based relative position embeddings, so we need to specify the PDB filename. This PDB file is in the [data/pdb_files](../data/pdb_files) directory, which the script checks by default, so there is no need to specify the full path. You can also just specify "auto" to use the PDB file defined for this dataset in [datasets.yml](../data/dms_data/datasets.yml).

```
--pdb_fn
1gfl_cm.pdb
```

## Network architecture arguments

For finetuning, we implemented a special model `transfer_model` that handles pretrained checkpoints with top nets. 
```
--model_name
transfer_model
```

The pretrained checkpoint can be a PyTorch checkpoint (.pt file) downloaded from the [metl-pretrained](https://github.com/gitter-lab/metl-pretrained) repository or a PyTorch Lightning checkpoint (.ckpt file) obtained from pretraining a model with this repository.
```
--pretrained_ckpt_path
pretrained_models/Hr4GNHws.pt
```

The backbone cutoff determines where to cutoff the pretrained model and place the new prediction head. For METL-Local models, we recommend backbone cutoff -1, and for METL-Global models we recommend backbone cutoff -2. 

```
--backbone_cutoff
-1
```

The remaining arguments determine the encoding, which should be set to `int_seqs`, whether to use dropout after the backbone cutoff, and the architecture of the new top net. You can leave these values as-is to match what we did for the preprint.

```
--encoding
int_seqs
--dropout_after_backbone
--dropout_after_backbone_rate
0.5
--top_net_type
linear
```

## Finetuning strategy arguments

We implemented a dual-phase finetuning strategy. During the first phase, the backbone weights are frozen and only the top net is trained. During the second phase, all the network weights are unfrozen and trained at a reduced learning rate.

The unfreeze_backbone_at_epoch argument determines the training epoch at which to unfreeze the backbone. We train the models for 500 epochs, so the backbone is unfrozen halfway through at epoch 250.

```
--finetuning
--finetuning_strategy
backbone
--unfreeze_backbone_at_epoch
250
--backbone_always_align_lr
```

## Optimization arguments

Basic optimizer arguments include the batch size, learning rate, and maximum number of epochs to train for. Unless early stopping is enabled, the model will train for the given number of epochs. 

```
--optimizer
adamw
--weight_decay
0.1
--batch_size
128
--learning_rate
0.001
--max_epochs
500
--gradient_clip_val
0.5
```

The learning rate scheduler we used for finetuning is a dual phase learning rate schedule that matches the dual phase finetuning strategy. Each phase has a linear learning rate warmup for 1% of the total steps in that phase. There is also a cosine decay for the learning rate for each phase. The phase 2 learning rate is 10% of the phase 1 learning rate.

```
--lr_scheduler
dual_phase_warmup_constant_cosine_decay
--warmup_steps
.01
--phase2_lr_ratio
0.1

```

## Loss Arguments

<u>Default</u>

By default, the loss function is **mean squared error (MSE)**:
```
--loss_func
mse
```
This does not need to be explicitly set in an argument file, but including it can aid reproducibility and help track different experimental runs.



#### Using Ordinal-Specific Loss

All ordinal loss functions require the target column for DMS finetuing to have labels 0, 1, ..., N-1 for N ordered classes, where 0 is the lowest and N-1 is the highest. If you need custom labels and want to do ordinal specific loss feel free to raise an issue on github.

<u>CORN</u>

If you are working with **ordinal data** (e.g., labels 0, 1, ..., N-1 for N ordered classes), consider using the ordinal-specific loss function (corn):

```
--loss_func
corn
--num_classes
4
--top_net_output_dim
3
--corn_coral_log_feature
3
```

This implements the CORN loss described in [this paper](https://arxiv.org/abs/2111.08851). The corn specific functions in *metl* are adapted from the [CORN+CORAL library](https://github.com/Raschka-research-group/coral-pytorch/tree/main).

- `--num_classes` specifies the number of classes (as an int).
- `--top_net_output_dim` output dimension from the final layer, must be number of classes minus 1 (as an int), to correspond with each probability task.
- `--corn_coral_log_feature` specifies which predicted class probability should be used in *metl*'s logging. This does **not** affect the final saved predictions, the probabilities of being in any class is saved, only the logged test metrics and plots.
  - Options: "raw_predictions", 2,..,N, where N is the maximum number of classes.
  - Note: `1` is not allowed, as the probability a variant is in the lowest class is always equal to 1.
  - For example, `3` logs the probability of the sample being in class 3.
  - By default, `corn_coral_log_feature` is set to `raw_predictions`, which converts class probabilities into the highest class with probability greater than 0.5.



<u>CORAL</u>
This will implement the loss and final layer described in [this paper](https://doi.org/10.1016/j.patrec.2020.11.008). The coral specific functions in *metl* are adapted from the [CORN+CORAL library](https://github.com/Raschka-research-group/coral-pytorch/tree/main).
```
--loss_func
coral
--top_net_type
coral
--preinit_bias
--num_classes
4
--corn_coral_log_feature
3

```


<u> CLASS IMBALANCE </u>

If you have ordinal data which has class imbalance, you could consider trying to weight the loss function to pay a greater penalty during training to classes which are not as represented using the flag:

```
--use_importance_weights
```

This will calculate the ratio of each class in the training set and weight them in the loss by the inverse of this ratio (if any class has zero samples the value of 0.1 is assigned to stop divide by zero errors.) The `--use_importance_weights` flag can be used with `--loss_func mse`, `--loss_func corn`, or `--loss_func coral`.

Alternatively, you can set your own weights using the flag
```
--set_importance_weights
0.3
0.3
0.4
```

If you are using the CORN or CORAL loss, what is weighted is each probability task, since there are only N-1 probability tasks, there must be N-1 weights (where N is the number of classes). For MSE loss N weights must be specified.


<u>MULTIPLE EXPERIMENTAL BATCHES</u>



or  . Other parameters exist to change the behavior of the loss functions,
such as `--use_importance_weights` which weights the proportion of each class equally to the loss function.






## Logging arguments

We have built in functionality for tracking model training with Weights & Biases. If you have a Weights and Biases account, set the environment variable `WANDB_API_KEY` to your API key and set the flag `--use_wandb` instead of `--no_use_wandb` below.

```
--no_use_wandb
--wandb_project
metl-target
--wandb_online
--experiment
default
```

The below argument determines where to place the log directory locally.
```
--log_dir_base
output/training_logs
```

# Running training

All the arguments described above are contained in [finetune_avgfp_local.txt](../args/pretrain_avgfp_local.txt), which can be fed directly into [train_target_model.py](train_target_model.py).

PyTorch Lightning has a built-in progress bar that is convenient for seeing training progress, but it does not display correctly in Jupyter when calling the script with `!python`. We are going to disable the progress bar for by setting the flag `--enable_progress_bar false`. Instead, we implemented a simple print statement to track training progress, which we will enable with the flag `--enable_simple_progress_messages`. 

The [train_target_model.py](../code/train_target_model.py) script can support running on Apple Silicon with acceleration via MPS, but the version of PyTorch used in this environment is slightly outdated and does not support all MPS operations, so MPS support has been disabled. The script will run on GPU via CUDA if available, otherwise it will use CPUs.

In [5]:
!python code/train_target_model.py @args/finetune_avgfp_local.txt --enable_progress_bar false --enable_simple_progress_messages --max_epochs 50 --unfreeze_backbone_at_epoch 25 --uuid inqx2jYi

Random seed not specified, using: 979104742
Global seed set to 979104742
User gave model UUID: inqx2jYi
Did not find existing log directory corresponding to given UUID: inqx2jYi
Created log directory: output/training_logs/inqx2jYi
Final UUID: inqx2jYi
Final log directory: output/training_logs/inqx2jYi
Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.rich_model_summary.RichModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Loading `train_dataloader` to estimate number of stepping batches.
/opt/homebrew/Caskroom/mambaforge/base/envs/metl/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1892: PossibleUserWarning: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you w

# Additional recommendations

## Model selection

Selecting the model from the epoch with the lowest validation set loss can help prevent overfitting. It requires having a big enough validation set that provides an accurate estimate of performance. 

We enabled model selection if the validation set size was ≥ 32 for METL-Local and ≥ 128 for METL-Global. We found the optimization was more stable for METL-Local than METL-Global, thus smaller validation sets were still reliable. 

Enable model selection by setting argument `--ckpt_monitor val`.


## Backbone cutoff for METL-Global
Finetuning METL-Global is largely the same as METL-Local, except we recommend using a different threshold for model selection (see above), as well as a different backbone cutoff.

For METL-Local, we set `--backbone_cutoff -1`, which attaches the new prediction head immediately after the final fully connected layer. 

For METL-Global, we recommend setting `--backbone_cutoff -2`, which attaches the new prediction head immediately after the global pooling layer. We found this resulted in better finetuning performance for METL-Global.

# Running inference using finetuned model

We provide two ways to run inference: using our PyTorch Lightning-based inference framework or using your own inference loop. The example below shows you would use your own inference loop. See the [inference.ipynb](inference.ipynb) notebook for additional information. 

In [6]:
import torch

import inference
import encode as enc
import utils  # for loading dataset metadata

In [9]:
# the Lightning checkpoint from the finetuning we performed above
ckpt_fn = "output/training_logs/inqx2jYi/checkpoints/epoch=49-step=50.ckpt"
model = inference.load_pytorch_module(ckpt_fn)

/Users/sg/PycharmProjects/metl/code/inference.py:159: UserWarning: Transforming checkpoint keys: strip_prefix='model.', add_prefix=''
  warnings.warn(


In [10]:
# load the GFP wild-type sequence and the PDB file (needed for 3D RPE)
datasets = utils.load_dataset_metadata()
wt = datasets["avgfp"]["wt_aa"]
wt_offset = datasets["avgfp"]["wt_ofs"]
pdb_fn = datasets["avgfp"]["pdb_fn"]

# some example GFP variants to compute the scores for
variants = ["E3K,G102S",
            "T36P,S203T,K207R",
            "V10A,D19G,F25S,E113V"]

# encode the variants into the integer sequences format
# to feed into the model
encoded_variants = enc.encode(
    encoding="int_seqs",
    variants=variants,
    wt_aa=wt,
    wt_offset=wt_offset,
    indexing="0_indexed"
)

# set model to eval mode
model.eval()

# no need to compute gradients for inference
with torch.no_grad():
    # note we are specifying the pdb_fn because this model uses 3D RPE
    predictions = model(torch.tensor(encoded_variants), pdb_fn=pdb_fn)

print(predictions)

tensor([[-0.3805],
        [-0.3623],
        [-0.2217]])
